# Fine-tune the model with the Amazon Customer Reviews Dataset and a set of prompts

In [ ]:
import psutil

notebook_memory = psutil.virtual_memory()
print(notebook_memory)

if notebook_memory.total < 32 * 1000 * 1000 * 1000:
    print('*******************************************')    
    print('YOU ARE NOT USING THE CORRECT INSTANCE TYPE')
    print('PLEASE CHANGE INSTANCE TYPE TO  m5.2xlarge ')
    print('*******************************************')
else:
    correct_instance_type=True

In [ ]:
#model_checkpoint = "facebook/opt-350m"
model_checkpoint = "bigscience/bloomz-560m"

# Create prompts for few-shot, one-shot, zero-shot inference on sample data

In [ ]:
import pandas as pd
import csv
file = './data-tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz'

# Read the file
df = pd.read_csv(file, delimiter="\t", quoting=csv.QUOTE_NONE, compression="gzip")

df.isna().values.any()
df = df.dropna()
df = df.reset_index(drop=True)    

print("Shape of dataframe {}".format(df.shape))

# Convert Pandas dataframes into Datasets
import datasets
from datasets import Dataset

# Create Dataset objects (Arrow PyTables) from Pandas dataframes
dataset = Dataset.from_pandas(df)

# Apply prompt    
from promptsource.templates import DatasetTemplates
prompt_templates = DatasetTemplates('amazon_us_reviews/Wireless_v1_00') 

for template in prompt_templates.templates.values():
    print(template.get_name())

prompt = prompt_templates["Given the review body return a categorical rating"]
print(prompt.answer_choices)    
print(prompt.__dict__)

dataset = dataset.select([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).map(lambda row : {'prompt': prompt.apply(row)[0], 'label': prompt.apply(row)[1]})
prompt = dataset[0]['prompt']
label = dataset[0]['label']
print(prompt)
print(label)

In [ ]:
prompt0 = dataset[0]
prompt1 = dataset[1]
prompt2 = dataset[2]
prompt3 = dataset[3]

few_shot_prompt = 'PROMPT: ' + prompt0['prompt'] + '\nRESPONSE: ' + prompt0['label'] + '\n\nPROMPT: ' + prompt1['prompt'] + '\nRESPONSE: ' + prompt1['label'] + '\n\nPROMPT: ' + prompt2['prompt'] + '\nRESPONSE: ' + prompt2['label'] + '\n\nPROMPT: ' + prompt3['prompt'] + '\nRESPONSE:'
one_shot_prompt = 'PROMPT: ' + prompt0['prompt'] + '\nRESPONSE: ' + prompt0['label'] + '\n\nPROMPT: ' + prompt1['prompt'] + '\nRESPONSE:'
zero_shot_prompt = 'PROMPT: ' + prompt0['prompt'] + '\nRESPONSE:'

# Perform few-shot, one-shot, zero-shot inference BEFORE fine-tuning

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

Now that the data has been loaded, we're ready to instantiate our `Trainer`. We will retrieve our pre-trained model:

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

# Few-shot

In [ ]:
inputs = tokenizer(few_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt3['label']))

# One-shot

In [ ]:
inputs = tokenizer(one_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt1['label']))

# Zero-shot

In [ ]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt0['label']))

# Fine-tune the model with the Amazon Customer Reviews Data

In [ ]:
from datasets import Dataset

lm_dataset_train = Dataset.from_parquet('./data/train/*.parquet')
print(lm_dataset_train.shape)

In [ ]:
from transformers import TrainingArguments
import torch

model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    f"{model_name}-finetuned-amazon-customer-reviews",
    learning_rate=2e-5,
    weight_decay=0.01, 
    max_steps=10,
    num_train_epochs=1.0,
    no_cuda=not torch.cuda.is_available()    
)

We pass along all of those to the `Trainer` class:

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset_train
)

In [ ]:
train_results = trainer.train()
train_results

# Save fine-tuned model

In [ ]:
model_path = './tmp_models/{}/'.format(model_checkpoint)

model.save_pretrained(model_path)

# Perform few-shot, one-shot, zero-shot inference AFTER fine-tuning

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(model_path)

This model also supports many advanced parameters while performing inference including the following:

**max_length**: Model generates text until the output length (which includes the input context length) reaches max_length. If specified, it must be a positive integer.

**num_return_sequences**: Number of output sequences returned. If specified, it must be a positive integer.

**num_beams**: Number of beams used in the greedy search. If specified, it must be integer greater than or equal to num_return_sequences.

**no_repeat_ngram_size**: Model ensures that a sequence of words of no_repeat_ngram_size is not repeated in the output sequence. If specified, it must be a positive integer greater than 1.

**temperature**: Controls the randomness in the output. Higher temperature results in output sequence with low-probability words and lower temperature results in output sequence with high-probability words. If temperature -> 0, it results in greedy decoding. If specified, it must be a positive float.

**early_stopping**: If True, text generation is finished when all beam hypotheses reach the end of stence token. If specified, it must be boolean.

**do_sample**: If True, sample the next word as per the likelyhood. If specified, it must be boolean.

**top_k**: In each step of text generation, sample from only the top_k most likely words. If specified, it must be a positive integer.

**top_p**: In each step of text generation, sample from the smallest possible set of words with cumulative probability top_p. If specified, it must be a float between 0 and 1.

**seed**: Fix the randomized state for reproducibility. If specified, it must be an integer.

In [ ]:
inputs = tokenizer(few_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt3['label']))

In [ ]:
inputs = tokenizer(one_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt1['label']))

In [ ]:
inputs = tokenizer(zero_shot_prompt, return_tensors='pt')

print(tokenizer.decode(model.generate(inputs["input_ids"], 
                       max_new_tokens=1,
                       do_sample=True, 
                       top_k=50, 
                       top_p=0.9
                      )[0]))

print('EXPECTED RESPONSE: {}'.format(prompt0['label']))

In [ ]:
import transformers
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer

print(model.get_memory_footprint())

quantized_model = AutoModelForCausalLM.from_pretrained(model_path, load_in_8bit=True, device_map='auto')

print(quantized_model.get_memory_footprint())


# EXTRAS (work-in-progress)

In [ ]:
# # Create Dataset objects (Arrow PyTables) from Pandas dataframes
# dataset = Dataset.from_pandas(df)

# # Apply prompt    
# from promptsource.templates import DatasetTemplates
# prompt_templates = DatasetTemplates('amazon_us_reviews/Wireless_v1_00') 

# for template in prompt_templates.templates.values():
#     print(template.get_name())

# prompt = prompt_templates["Given the review body return a categorical rating"]
# print(prompt.answer_choices)    
# print(prompt.__dict__)

# dataset = dataset.select([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]).map(lambda row : {'prompt': prompt.apply(row)[0], 'label': prompt.apply(row)[1]})
# prompt = dataset[0]['prompt']
# label = dataset[0]['label']
# print(prompt)
# print(label)

In [ ]:
# prompt0 = dataset[0]
# prompt1 = dataset[1]
# prompt2 = dataset[2]
# prompt3 = dataset[3]

# #few_shot_prompt = prompt0['prompt'] + '\nFor the previous review, is the sentiment positive, neutral, or negative?' + prompt0['label'] + '\n\nPROMPT: ' + prompt1['prompt'] + '\nRESPONSE: ' + prompt1['label'] + '\n\nPROMPT: ' + prompt2['prompt'] + '\nRESPONSE: ' + prompt2['label'] + '\n\nPROMPT: ' + prompt3['prompt'] + '\nRESPONSE:'
# #one_shot_prompt = 'PROMPT: ' + prompt0['prompt'] + '\nRESPONSE: ' + prompt0['label'] + '\n\nPROMPT: ' + prompt1['prompt'] + '\nRESPONSE:'
# zero_shot_prompt = prompt0['prompt'] + '\nFor the previous review, is the sentiment positive, neutral, or negative?'

In [ ]:
# inputs = tokenizer(zero_shot_prompt, return_tensors='pt')

# print(tokenizer.decode(model.generate(inputs["input_ids"], 
#                        max_new_tokens=1,
#                        do_sample=True, 
#                        top_k=50, 
#                        top_p=0.9
#                       )[0]))

# print('EXPECTED RESPONSE: {}'.format(prompt0['label']))

In [ ]:
# %%html

# <p><b>Shutting down your kernel for this notebook to release resources.</b></p>
# <button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
# <script>
# try {
#     els = document.getElementsByClassName("sm-command-button");
#     els[0].click();
# }
# catch(err) {
#     // NoOp
# }    
# </script>